In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import openpyexcel
from openpyexcel import workbook, load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyexcel.utils import get_column_letter
from itertools import islice
from os import listdir
import re
import csv
import datetime
import pickle

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:.0f}'.format

In [3]:
cash_sheets = pd.read_pickle('pickle/cash_sheets.pkl')


In [4]:
cash_map_lookup = pd.read_pickle('pickle/cash_map_lookup.pkl')

In [5]:
cash_map_lookup

{'heading': {'Adjustments to reconcile consolidated net earnings to net cash provided by operating activities:': 'ARNI',
  'Investing Activities': 'CIA',
  'Cash flows from operating activities:': 'CFOA',
  'Cash Flows from Financing Activities': 'CFA',
  'Operating activities:': 'CFOA',
  'Adjustments to arrive at cash provided by operating activities:': 'ARNI',
  'CASH PROVIDED BY / (USED IN) OPERATING ACTIVITIES': 'CFOA',
  'Significant Non-cash Investing and Financing Activity': 'CFA',
  'Changes in Operating Assets and Liabilities [Abstract]': 'COAL',
  'Adjustments to reconcile net income (loss) to net cash flows from operating activities:': 'ARNI',
  'Changes in operating assets and liabilities net of effects from mergers and disposals of businesses:': 'COAL',
  'Cash flows used in investing activities:': 'CIA',
  'Cash Flows from Operating Activities': 'CFOA',
  'Adjustments to reconcile net income to net cash from operating activities:': 'ARNI',
  'Changes in operating assets 

In [6]:
cash_table_map = {'CFOA': ['NI'],
                  'ARNI': ['SC'],
                  'COAL': ['NCO'],
                  'CIA':  ['PPE','CE','NCI'],
                  'CFA':  ['SBB','D','NCF'],
                  'No Heading': ['CEY']}

In [7]:
cash_sheets['CAT'][1]

,2020,2019,2018
Index,,,
Cash flow from operating activities:,NaN,NaN,NaN
Profit of consolidated and affiliated companies,3003,6094,6148
Adjustments for non-cash items:,NaN,NaN,NaN
Depreciation and amortization,2432,2577,2766
Actuarial loss on pension and postretirement benefits,383,468,495
Provision for deferred income taxes,-74,28,220
Other,1000,675,1006
"Changes in assets and liabilities, net of acquisitions and divestitures:",NaN,NaN,NaN
Receivables - trade and other,1442,171,-1619


In [10]:
def find_line_items(symbol, table, cash_map_lookup):
  
  cash_table_map = {'CFOA': ['NI'],
                    'ARNI': ['SC'],
                    'COAL': ['NCO'],
                    'CIA':  ['PPE','CE','NCI'],
                    'CFA':  ['SBB','D','NCF'],
                    'No Heading': ['CEY']}
    
  heading_lookup = cash_map_lookup['heading']
  line_item_lookup = cash_map_lookup['line_item']

  df = pd.DataFrame()
  h = 'No Heading'
  line_item_list = []
  items_found = []
  items_required = ['NI','SC','NCO','PPE','CE','NCI','SBB','D','NCF','CEY']
    
  # First try and find line item under assumed heading. If not found then look for a match across whole table
                    
  # Go through each line in table, if row contains only nan then assume heading

  for index, row in table.iterrows():
   
    if row.isnull().all():
        
      if index in heading_lookup:
        h = heading_lookup[index]
        line_item_list = cash_table_map[h]
        df = df.append(row)
  
    else:
      # Assume that row is not a header
       
      if index in line_item_lookup:
        l = line_item_lookup[index]
        items_found.append(l)
            
        #print(row)
        df = df.append(row)
                    
  # Determine items not found and do a search for each item ignoring headings
                    
  outstanding_items = set(items_required) - set(items_found)

  for item in outstanding_items:
                    
    #Iterate through each row in table and look for match
                    
     for index, row in table.iterrows():
    
       # Skip row if header otherwise look for match for line item
        
       if ~row.isnull().all():
       
         if index in line_item_lookup:
            l = line_item_lookup[index]
            
            if l in outstanding_items:
              print('')
              print('<< Outstanding Item >> ',l)     
              #print(index)
              
              r1 = pd.Series(l, index=['type'],name=index)
              r2 = pd.concat([row,r1])
              df = df.append(row)
                
              outstanding_items.remove(l)     
                                    
    # TBD FInd line items still missing by doing a scan through table and finding first match 
    
    # Aggregate line items with same type
    
  print("item found:", items_found)

  return(df)
    

In [25]:
k = find_line_items('CAT',cash_sheets['CAT'][1], cash_map_lookup)
k

item found: ['NCO', 'D']


,2020,2019,2018
Cash flow from operating activities:,NaN,NaN,NaN
Net Cash Provided by Operating Activities,6327,6912,6558
Cash flow from investing activities:,NaN,NaN,NaN
Cash flow from financing activities:,NaN,NaN,NaN
Dividends paid,-2243,-2132,-1951


In [51]:
l = ['a','b','c']
k = pd.Series(l, index=['v1','v2','v3'])
k1 = pd.Series('d', index=['type'])
r = k.append(k1)
r

v1      a
v2      b
v3      c
type    d
dtype: object

In [32]:
def build_sheet(ticker, table_list, cash_map_lookup):
    
  sheet = pd.DataFrame()

  for i in range(0,len(table_list)):
   new_sheet = find_line_items(ticker, table_list[i], cash_map_lookup)
   sheet = pd.merge(sheet,new_sheet, left_index=True, right_index=True, how='outer')
                         
  col = sheet.columns
  r = re.compile(".*_[xy]$")
  x = list(filter(r.match, col)) 
  sheet = sheet.drop(columns=x)

  return sheet

In [33]:
build_sheet('CAT',cash_sheets['CAT'],cash_map_lookup)


item found: ['NCO', 'D']
item found: ['NCO', 'D']
item found: ['NCO', 'NCI', 'D', 'NCF']
item found: ['NCO', 'NCI', 'D', 'D', 'NCF']
item found: ['NCO', 'NCI', 'D', 'D', 'NCF']


,2022,2021,2019,2017,2015,2013,2012
Cash flow from financing activities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash flow from financing activities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash flow from financing activities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash flow from investing activities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash flow from operating activities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dividends paid,-2440,-2332,-2132,-1831,-1757,-1111,-1617
Excess tax benefit from stock-based compensation,NaN,NaN,NaN,NaN,24,96,192
Net Cash Provided by Operating Activities,7766,7198,6912,NaN,NaN,NaN,NaN
Net cash provided by financing activities,NaN,NaN,NaN,-3657,-3870,-4511,3606
Net cash provided by investing activities,NaN,NaN,NaN,-966,-3517,-5046,-6190


In [24]:
cash_sheets['CAT'][1]

,2020,2019,2018
Index,,,
Cash flow from operating activities:,NaN,NaN,NaN
Profit of consolidated and affiliated companies,3003,6094,6148
Adjustments for non-cash items:,NaN,NaN,NaN
Depreciation and amortization,2432,2577,2766
Actuarial loss on pension and postretirement benefits,383,468,495
Provision for deferred income taxes,-74,28,220
Other,1000,675,1006
"Changes in assets and liabilities, net of acquisitions and divestitures:",NaN,NaN,NaN
Receivables - trade and other,1442,171,-1619


In [126]:
# Iterate through cash_sheets

result = {}

for ticker, table_list in cash_sheets.items():
  print(key)
  sheet = build_sheet(ticker, table_list, cash_map_lookup)
  result[ticker] = sheet


WYNN
item found: ['NI', 'SC', 'NCO', 'PPE', 'NCI', 'NCF', 'CEY']
item found: ['NI', 'SC', 'NCO', 'PPE', 'NCI', 'NCF', 'CEY']
item found: ['NI', 'SC', 'D', 'NCO', 'PPE', 'NCI', 'D', 'NCF', 'CEY']
item found: ['NI', 'SC', 'D', 'D', 'NCO', 'PPE', 'NCI', 'D', 'NCF', 'CEY']
item found: ['NI', 'SC', 'D', 'NCO', 'PPE', 'NCI', 'D', 'NCF', 'CEY']
WYNN
item found: ['SC', 'CE', 'D', 'PPE']
item found: ['PPE', 'SC', 'PPE', 'D']
item found: ['SC', 'PPE', 'D']
item found: ['SC', 'PPE', 'D']
item found: ['SC', 'PPE', 'D', 'CEY']
WYNN
item found: ['NI', 'SC', 'PPE', 'D']
item found: ['NI', 'SC', 'PPE', 'D']
item found: ['NI', 'SC', 'PPE', 'D']
item found: ['NI', 'SC', 'PPE', 'D']
item found: ['NI', 'SC', 'PPE', 'D']
WYNN
item found: ['NI', 'SC', 'NCO', 'PPE', 'NCI', 'D', 'NCF', 'CEY']
item found: ['NI', 'SC', 'NCO', 'PPE', 'NCI', 'D', 'NCF', 'CEY']
item found: ['NI', 'SC', 'NCO', 'PPE', 'NCI', 'D', 'NCF', 'CEY']
item found: ['NI', 'SC', 'NCO', 'PPE', 'NCI', 'D', 'NCF']
item found: ['NI', 'SC', 'PPE', 

item found: ['SC', 'D', 'NCO', 'PPE', 'D', 'D', 'NCF', 'CEY']
item found: ['SC', 'D', 'NCO', 'PPE', 'NCI', 'D', 'D', 'NCF', 'CEY']
WYNN
item found: ['SC', 'NCO', 'CE', 'NCI', 'D', 'NCF']
item found: ['SC', 'NCO', 'CE', 'NCI', 'D', 'NCF']
item found: ['SC', 'NCO', 'CE', 'NCI', 'D', 'NCF']
item found: ['SC', 'CE', 'NCI', 'D', 'D', 'NCF']
item found: ['SC', 'CE', 'NCI', 'D', 'D', 'NCF']
WYNN
item found: ['NI', 'SC', 'PPE', 'D', 'CEY', 'PPE', 'PPE']
item found: ['NI', 'SC', 'PPE', 'D', 'CEY', 'PPE', 'PPE']
item found: ['NI', 'SC', 'PPE', 'D', 'CEY', 'PPE', 'PPE']
item found: ['NI', 'SC', 'PPE', 'D', 'CEY', 'PPE', 'PPE']
item found: ['NI', 'SC', 'PPE', 'D', 'CEY', 'PPE', 'PPE']
WYNN
item found: ['NI', 'SC', 'NCO', 'CE', 'NCI', 'D', 'NCF']
item found: ['NI', 'SC', 'NCO', 'CE', 'NCI', 'D', 'NCF']
item found: ['NI', 'SC', 'NCO', 'CE', 'NCI', 'D', 'NCF']
item found: ['NI', 'SC', 'NCO', 'CE', 'NCI', 'D', 'NCF']
item found: ['NI', 'SC', 'NCO', 'CE', 'NCI', 'D', 'NCF']
WYNN
item found: ['NI', 'NCO

In [136]:
result['NKE']

,2022,2021,2019,2017,2015,2013,2012
"Additions to property, plant and equipment",-758,-695,-1119,-1105,-963,-598,-563
"CASH AND EQUIVALENTS, END OF YEAR",8574,9889,4466,3808,3852,3337,2254
Dividends â€” common and preferred,NaN,NaN,NaN,NaN,NaN,-703,-619
Dividends — common and preferred,-1837,-1638,-1332,-1133,-899,NaN,NaN
"Interest, net of capitalized interest",290,293,153,98,53,20,29
Net income,6046,5727,4029,4240,3273,2472,2211
"Non-cash additions to property, plant and equipment",160,179,160,266,206,137,99
Stock-based compensation,638,611,325,215,191,174,130


In [17]:
cash_map_lookup

{'heading': {'Adjustments to reconcile consolidated net earnings to net cash provided by operating activities:': 'ARNI',
  'Investing Activities': 'CIA',
  'Cash flows from operating activities:': 'CFOA',
  'Cash Flows from Financing Activities': 'CFA',
  'Operating activities:': 'CFOA',
  'Adjustments to arrive at cash provided by operating activities:': 'ARNI',
  'CASH PROVIDED BY / (USED IN) OPERATING ACTIVITIES': 'CFOA',
  'Significant Non-cash Investing and Financing Activity': 'CFA',
  'Changes in Operating Assets and Liabilities [Abstract]': 'COAL',
  'Adjustments to reconcile net income (loss) to net cash flows from operating activities:': 'ARNI',
  'Changes in operating assets and liabilities net of effects from mergers and disposals of businesses:': 'COAL',
  'Cash flows used in investing activities:': 'CIA',
  'Cash Flows from Operating Activities': 'CFOA',
  'Adjustments to reconcile net income to net cash from operating activities:': 'ARNI',
  'Changes in operating assets 

In [115]:
col = sheet.columns
r = re.compile(".*_[xy]$")
x = list(filter(r.match, col)) 
sheet.drop(columns=x)

,2023,2022,2020,2018,2016,2014,2013
Cash and cash equivalents at end of year,NaN,NaN,NaN,NaN,8705,7281,7781
"Cash, cash equivalents and restricted cash at end of year",8841,14834,9515,NaN,NaN,NaN,NaN
Consolidated net income,11292,13940,15201,10523,15080,16695,17756
Dividends paid,-6114,-6152,-6048,-6124,-6294,-6139,-5361
Net cash provided by operating activities,28841,24181,25255,28337,27389,23257,25591
Net cash used in financing activities,-17039,-22828,-14299,-19875,-16122,-10789,-11946
Net cash used in investing activities,-17722,-6015,-9128,-9079,-10675,-12526,-12637
Payments for property and equipment,-16857,-13106,-10705,-10051,-11477,-13115,-12898


In [25]:
heading_lookup = cash_map_lookup['heading']
heading_lookup

{'Adjustments to reconcile consolidated net earnings to net cash provided by operating activities:': 'ARNI',
 'Investing Activities': 'CIA',
 'Cash flows from operating activities:': 'CFOA',
 'Cash Flows from Financing Activities': 'CFA',
 'Operating activities:': 'CFOA',
 'Adjustments to arrive at cash provided by operating activities:': 'ARNI',
 'CASH PROVIDED BY / (USED IN) OPERATING ACTIVITIES': 'CFOA',
 'Significant Non-cash Investing and Financing Activity': 'CFA',
 'Changes in Operating Assets and Liabilities [Abstract]': 'COAL',
 'Adjustments to reconcile net income (loss) to net cash flows from operating activities:': 'ARNI',
 'Changes in operating assets and liabilities net of effects from mergers and disposals of businesses:': 'COAL',
 'Cash flows used in investing activities:': 'CIA',
 'Cash Flows from Operating Activities': 'CFOA',
 'Adjustments to reconcile net income to net cash from operating activities:': 'ARNI',
 'Changes in operating assets and liabilities, net of ac

In [23]:
cash_map_lookup['line_item']

{'Net cash generated from operating activities': 'NCO',
 'Payment of dividends': 'D',
 'Change in accounts and construction payables related to property and equipment': 'PPE',
 'Cash dividends': 'D',
 'Net cash used in operating activities': 'NCO',
 'Net cash flows (used in) financing activities': 'NCF',
 'Net cash provided by (used in) financing activities': 'NCF',
 'Net Cash Used in Investing Activities': 'NCI',
 'Net cash flows used in investing activities': 'NCI',
 'Net cash provided by (used for) operating activities': 'NCO',
 'Net cash used for financing activities from continuing operations': 'NCF',
 'Net cash provided by (used for) investing activities from continuing operations': 'NCI',
 'Excess tax benefit from share-based compensation': 'D',
 'Non-cash property additions': 'PPE',
 'Net cash used in investing activities from continuing operations': 'NCI',
 'Acquisitions of property and equipment included in liabilities': 'PPE',
 'Net Cash (Used for)/Provided by Investing Acti

In [30]:
table

,2020,2019,2018
,,,
Cash flows from operating activities:,NaN,NaN,NaN
Net income,5260,2951,2591
Adjustments to reconcile net income to net cash provided by operating activities:,NaN,NaN,NaN
"Depreciation, amortization and accretion",757,757,346
Stock-based compensation,909,788,610
Reduction of operating lease right-of-use assets,87,0,0
Deferred income taxes,-1501,3,-469
"Unrealized losses (gains) on investments, net",-11,-48,1
Other non-cash items,40,14,7


In [19]:
def split_table(t):
  l = []
  headings = []
  df = pd.DataFrame()

  for index, row in t.iterrows():
    
    if row.replace('', np.nan).isna().all():
      # Blank row, start new DataFrame
    
      headings.append(row)
      l.append(df)
      df = pd.DataFrame()
    df = df.append(row)

  return l, headings


In [128]:
for key, value in cash_sheets.items():
  print(key)

ADBE
ADM
AME
AMGN
ANET
BBY
BG
BIIB
CAT
CHTR
CMCSA
COST
CPB
CSCO
DG
DLTR
FFIV
GILD
GM
INCY
JNPR
KO
KR
LNT
LVS
MDLZ
MGM
MLM
MSI
NKE
ORCL
PCAR
PEG
PEP
REGN
RL
ROK
ROST
TAP
TGT
TJX
TSLA
VZ
WAB
WEC
WMT
WYNN
